In [1]:
import z3 as z
z.set_param('parallel.enable', False)

In [2]:
n = 3
N = 1050

In [3]:
ints = [z.Int(f'int_{i}') for i in range(n)]

In [4]:
s = z.Solver()

for i in range(n-1):
    # Constraint #1
    s.add(ints[i] > 0)
    # Constraint #2
    s.add(ints[i+1] == ints[i] + 1)

# Constraint #3
s.add(z.Sum(ints) == N)

In [5]:
if s.check() == z.sat:
    m = s.model()
    print('Found a solution')
    print(f'({n} terms) {m[ints[0]].as_long()} + ... + {m[ints[n -1]].as_long()} == {N}')
else:
    print(f'Finding {n} consecutive integers with a sum of {N} is unsatisfiable')

Found a solution
(3 terms) 349 + ... + 351 == 1050


In [6]:
def build_solver(n: int, N: int):
    # Create symbolic variables
    ints = [z.Int(f'int{i}') for i in range(n)]
    
    # Instantiate a solver
    s = z.Solver()
    
    for i in range(n-1):
        # Consecutive numbers
        s.add(ints[i+1] == ints[i] + 1)
        # Non-zero natural numbers
        s.add(ints[i] > 0)
    
    # Sum
    s.add(z.Sum(ints) == N)
    
    return s

In [7]:
s = build_solver(n = 3, N = 1337)
print(s)

[int1 == int0 + 1,
 int0 > 0,
 int2 == int1 + 1,
 int1 > 0,
 int0 + int1 + int2 == 1337]


Using this approach, we can check if sets of model inputs are satisfiable. For example, below we're exploring the solution space for N = 1337:

In [8]:
for i in range(3, 10):
    print(f'Sum 1337 is {build_solver(n = i, N = 1337).check()} with {i} integers')

Sum 1337 is unsat with 3 integers
Sum 1337 is unsat with 4 integers
Sum 1337 is unsat with 5 integers
Sum 1337 is unsat with 6 integers
Sum 1337 is sat with 7 integers
Sum 1337 is unsat with 8 integers
Sum 1337 is unsat with 9 integers


In [9]:
output_prev = z.BitVec('output_prev', 32)
states = {f'state_{i}': z.BitVec(f'state_{i}', 32) for i in range(1, 11)}
output_next = z.BitVec('output_next', 32)

print(type(states['state_1']))

<class 'z3.z3.BitVecRef'>


In [10]:
s = z.Solver()

In [11]:
for i in range(2, 11):
    s.add(states[f'state_{i}'] == states[f'state_{i - 1}'] * 214013 + 2531011)

print(s)

[state_2 == state_1*214013 + 2531011,
 state_3 == state_2*214013 + 2531011,
 state_4 == state_3*214013 + 2531011,
 state_5 == state_4*214013 + 2531011,
 state_6 == state_5*214013 + 2531011,
 state_7 == state_6*214013 + 2531011,
 state_8 == state_7*214013 + 2531011,
 state_9 == state_8*214013 + 2531011,
 state_10 == state_9*214013 + 2531011]


In [12]:
random_nums = [4, 54, 63, 79, 13, 55, 76, 11, 14, 45]

for i in range(2, 10):
    s.add(z.URem((states[f'state_{i}'] >> 16) & 0x7FFF ,100) == random_nums[i - 1])

In [13]:
s.add(output_prev == z.URem((states['state_1'] >> 16) & 0x7FFF ,100))
s.add(output_next == z.URem((states['state_10'] >> 16) & 0x7FFF ,100))

In [14]:
print(s)

[state_2 == state_1*214013 + 2531011,
 state_3 == state_2*214013 + 2531011,
 state_4 == state_3*214013 + 2531011,
 state_5 == state_4*214013 + 2531011,
 state_6 == state_5*214013 + 2531011,
 state_7 == state_6*214013 + 2531011,
 state_8 == state_7*214013 + 2531011,
 state_9 == state_8*214013 + 2531011,
 state_10 == state_9*214013 + 2531011,
 URem(state_2 >> 16 & 32767, 100) == 54,
 URem(state_3 >> 16 & 32767, 100) == 63,
 URem(state_4 >> 16 & 32767, 100) == 79,
 URem(state_5 >> 16 & 32767, 100) == 13,
 URem(state_6 >> 16 & 32767, 100) == 55,
 URem(state_7 >> 16 & 32767, 100) == 76,
 URem(state_8 >> 16 & 32767, 100) == 11,
 URem(state_9 >> 16 & 32767, 100) == 14,
 output_prev == URem(state_1 >> 16 & 32767, 100),
 output_next == URem(state_10 >> 16 & 32767, 100)]


In [15]:
print(s.check())

sat


In [16]:
print(s.model())

[state_3 = 3311639122,
 state_7 = 535860406,
 state_2 = 1700980091,
 state_4 = 4092565453,
 state_8 = 1173829793,
 state_5 = 2417052508,
 state_6 = 3389729967,
 state_1 = 2436150040,
 state_9 = 2200877280,
 output_next = 45,
 output_prev = 4,
 state_10 = 173405219]


In [17]:
def break_rand(nums: list, next_num: int):
    n_nums = len(nums)
    #print(f'len nums: {n_nums}')
    
    states = {f'state_{i}': z.BitVec(f'state_{i}', 32) for i in range(1, n_nums + 2)}
    #print(states)
    output_next = z.BitVec('output_next', 32)
    
    s = z.Solver()
    
    for i in range(2, n_nums + 2):
        s.add(states[f'state_{i}'] == states[f'state_{i - 1}'] * 214013 + 2531011)
        
    for i in range(1, n_nums + 1):
        s.add(z.URem((states[f'state_{i}'] >> 16) & 0x7FFF ,100) == nums[i - 1])
        
    s.add(output_next == z.URem((states[f'state_{n_nums + 1}'] >> 16) & 0x7FFF ,100))
    
    #print(s)
    
    if s.check() == z.sat:
        print(f'For the sequence: {nums}, problem is satisfiable')
        print(f'We were expecting: {next_num} and got: {s.model()[output_next]}\n')
    else:
        print(f'For the sequence: {nums}, problem is unsatisfiable')
    
    return s, states, output_next

In [18]:
random_nums = [4, 54, 63, 79, 13, 55, 76, 11, 14, 45]

In [19]:
for i in range(3, 10):
    break_rand(random_nums[:i], random_nums[i])

For the sequence: [4, 54, 63], problem is satisfiable
We were expecting: 79 and got: 94

For the sequence: [4, 54, 63, 79], problem is satisfiable
We were expecting: 13 and got: 60

For the sequence: [4, 54, 63, 79, 13], problem is satisfiable
We were expecting: 55 and got: 55

For the sequence: [4, 54, 63, 79, 13, 55], problem is satisfiable
We were expecting: 76 and got: 76

For the sequence: [4, 54, 63, 79, 13, 55, 76], problem is satisfiable
We were expecting: 11 and got: 11

For the sequence: [4, 54, 63, 79, 13, 55, 76, 11], problem is satisfiable
We were expecting: 14 and got: 14

For the sequence: [4, 54, 63, 79, 13, 55, 76, 11, 14], problem is satisfiable
We were expecting: 45 and got: 45



In [20]:
def enumerate_solutions(nums: list, next_num: int, print_model: bool = False, print_solutions: bool = False):
    s, states, output_next = break_rand(nums = nums, next_num = next_num)
    
    counter = 0
    solution_list = []
     
    while s.check() == z.sat:
        counter += 1
        solution_list.append(s.model()[output_next].as_long())
        
        print(f'Solution #{counter}')
        if print_model:
            print(f'{s.model()}\n')

        # Create constraints using string concatenation
        or_expression = 'z.Or('
        for i in states.keys():
            if i == 'output_next': continue
            or_expression += f'states[i] != s.model()[states[i]], '
        or_expression += ')'
        
        s.add(eval(or_expression))
        
    print(f'Found a total of {counter} solutions')
    if print_solutions:
        print(f'The solutions are: \n{solution_list}')
        print(f'The solution set is: \n{set(solution_list)}')

In [21]:
enumerate_solutions(random_nums[:5], random_nums[5], print_model = True)

For the sequence: [4, 54, 63, 79, 13], problem is satisfiable
We were expecting: 55 and got: 55

Solution #1
[state_3 = 1164155474,
 state_2 = 3848463739,
 state_1 = 288666392,
 state_4 = 1945081805,
 state_5 = 269568860,
 output_next = 55,
 state_6 = 1242246319]

Solution #2
[state_3 = 3311639122,
 state_2 = 1700980091,
 state_1 = 2436150040,
 state_4 = 4092565453,
 output_next = 55,
 state_5 = 2417052508,
 state_6 = 3389729967]

Found a total of 2 solutions


In [22]:
enumerate_solutions(random_nums[:9], random_nums[9], print_model=True)

For the sequence: [4, 54, 63, 79, 13, 55, 76, 11, 14], problem is satisfiable
We were expecting: 45 and got: 45

Solution #1
[state_7 = 2683344054,
 state_2 = 3848463739,
 state_1 = 288666392,
 state_4 = 1945081805,
 state_8 = 3321313441,
 state_3 = 1164155474,
 state_5 = 269568860,
 state_6 = 1242246319,
 state_9 = 53393632,
 output_next = 45,
 state_10 = 2320888867]

Solution #2
[state_7 = 535860406,
 state_2 = 1700980091,
 state_1 = 2436150040,
 state_4 = 4092565453,
 state_9 = 2200877280,
 state_8 = 1173829793,
 state_3 = 3311639122,
 state_10 = 173405219,
 output_next = 45,
 state_5 = 2417052508,
 state_6 = 3389729967]

Found a total of 2 solutions


In [23]:
enumerate_solutions(random_nums[:4], random_nums[4], print_solutions=True)

For the sequence: [4, 54, 63, 79], problem is satisfiable
We were expecting: 13 and got: 60

Solution #1
Solution #2
Solution #3
Solution #4
Solution #5
Solution #6
Solution #7
Solution #8
Solution #9
Solution #10
Solution #11
Solution #12
Solution #13
Solution #14
Solution #15
Solution #16
Solution #17
Solution #18
Solution #19
Solution #20
Solution #21
Solution #22
Solution #23
Solution #24
Solution #25
Solution #26
Solution #27
Solution #28
Solution #29
Solution #30
Solution #31
Solution #32
Solution #33
Solution #34
Solution #35
Solution #36
Solution #37
Solution #38
Solution #39
Solution #40
Found a total of 40 solutions
The solutions are: 
[60, 49, 49, 9, 9, 29, 29, 32, 32, 18, 18, 93, 93, 69, 77, 77, 69, 67, 5, 5, 25, 25, 13, 13, 55, 55, 48, 48, 12, 12, 60, 24, 67, 24, 16, 16, 89, 89, 96, 96]
The solution set is: 
{5, 9, 12, 13, 16, 18, 24, 25, 29, 32, 48, 49, 55, 60, 67, 69, 77, 89, 93, 96}


In [24]:
enumerate_solutions([0, 0, 0, 0], 0)

For the sequence: [0, 0, 0, 0], problem is satisfiable
We were expecting: 0 and got: 10

Solution #1
Solution #2
Solution #3
Solution #4
Solution #5
Solution #6
Solution #7
Solution #8
Solution #9
Solution #10
Solution #11
Solution #12
Solution #13
Solution #14
Solution #15
Solution #16
Solution #17
Solution #18
Solution #19
Solution #20
Solution #21
Solution #22
Solution #23
Solution #24
Solution #25
Solution #26
Solution #27
Solution #28
Solution #29
Solution #30
Solution #31
Solution #32
Solution #33
Solution #34
Solution #35
Solution #36
Solution #37
Solution #38
Found a total of 38 solutions


In [25]:
enumerate_solutions([0, 0, 0, 0, 0], 0)

For the sequence: [0, 0, 0, 0, 0], problem is unsatisfiable
Found a total of 0 solutions


__Conclusion__: based on these two experiments, there are 38 initial states (seeds) that result in four successive zeros, while there is __no__ seed that produces five successive zeros.

In [26]:
try:
    import bitstring as bs
except ModuleNotFoundError:
    !pip install bitstring
    import bitstring as bs

In [27]:
def make_constraints_next(n_constraints: int, slope: int = 0x5DEECE66D, intercept: int = 0xB, gen_bits = 31):
    # Define some constants
    addend = z.BitVecVal(intercept, 64)
    multiplier = z.BitVecVal(slope, 64)
    mask = z.BitVecVal((1 << 48) - 1, 64)
    
    # Define symbolic variables for the seed variable
    seeds = {f'seed_{i}': z.BitVec(f'seed_{i}', 64) for i in range(n_constraints)}

    constraints = []
    
    # Build constraints for the relation in row 175
    for i in range(1, n_constraints):
        constraints.append(seeds[f'seed_{i}'] == z.simplify((seeds[f'seed_{i-1}'] * multiplier + addend) & mask))
        
    # Define symbolic variables for the output from next()
    next_outputs = {f'next_output_{i}': z.BitVec(f'output{i}', 32) for i in range(1, n_constraints)}
    
    # Build the constraints for the relation in row 176
    for i in range(1, n_constraints):
        constraints.append(next_outputs[f'next_output_{i}'] == z.simplify(z.Extract(31, 0, z.LShR(seeds[f'seed_{i}'], 48 - gen_bits))))
        
    return constraints, seeds, next_outputs


In [28]:
def find_seed(sequence_length: int, slope: int = 0x5DEECE66D, intercept: int = 0xB):
    # Define some constants
    addend = z.BitVecVal(intercept, 64)
    multiplier = z.BitVecVal(slope, 64)
    mask = z.BitVecVal((1 << 48) - 1, 64)
    
    # Note we're generating an extra constraint
    # This is required since we'll be using seed_0 to extract the Random() instantiation value
    next_constraints, seeds, next_outputs = make_constraints_next(n_constraints=sequence_length+1, gen_bits=32)
    
    # Define a symbolic variable that we'll use to get the value that instantiated Random()
    original_seed = z.BitVec('original_seed', 64)
    
    # Build a solver object
    s = z.Solver()
    
    # Build a constraint that relates seed_0 and the value used to instantiate Random()
    s.add(seeds[f'seed_0'] == (original_seed ^ multiplier) & mask)
    
    # Next let's add the constraints we've built for next()
    s.add(next_constraints)
    
    # Lastly, let's return all the objects we've constructed so far   
    return s, original_seed, seeds, next_outputs

In [29]:
known_ints = [-1460590454, 747279288, -1334692577, -539670452, -501340078, -143413999]

In [30]:
solver, original_seed, seeds, next_ouputs = find_seed(sequence_length=len(known_ints))

# Notice: we setup the constraints so that next_outputs_1 is the result of seed_1 since we're using seed_0 for other uses
# Consequently, the index in known_ints is smaller by 1 than the index for next_outputs
solver.add(next_ouputs[f'next_output_{1}'] == known_ints[0])
solver.add(next_ouputs[f'next_output_{2}'] == known_ints[1])

# Lets take a look at our constraints before trying to solve them
print(solver)

[seed_0 == (original_seed ^ 25214903917) & 281474976710655,
 seed_1 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_0)),
 seed_2 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_1)),
 seed_3 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_2)),
 seed_4 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_3)),
 seed_5 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_4)),
 seed_6 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_5)),
 output1 == Extract(47, 16, seed_1),
 output2 == Extract(47, 16, seed_2),
 output3 == Extract(47, 16, seed_3),
 output4 == Extract(47, 16, seed_4),
 output5 == Extract(47, 16, seed_5),
 output6 == Extract(47, 16, seed_6),
 output1 == 2834376842,
 output2 == 747279288]


In [31]:
solver.check()

sat

In [32]:
solver.model()[original_seed]

1337

In [33]:
for i, known_int in enumerate(known_ints):
    calculated_int = solver.model()[next_ouputs[f'next_output_{i+1}']].as_long()
    assert bs.BitArray(uint=calculated_int, length=32).int == known_int
print('All assertions passed')

All assertions passed


In [34]:
# Generate all possible index pair combinations for known_ints
import itertools
index_pairs = itertools.combinations(range(len(known_ints)), 2)

# Now let's run our algorithm on each pair
for i, j in index_pairs:
    print(f'Trying sequence indices: {i}, {j}')
    # Generate a new solver object
    solver, original_seed, seeds, next_ouputs = find_seed(sequence_length=len(known_ints))

    # Set constraints for the output
    solver.add(next_ouputs[f'next_output_{i+1}'] == known_ints[i])
    solver.add(next_ouputs[f'next_output_{j+1}'] == known_ints[j])

    assert solver.check() == z.sat
    assert solver.model()[original_seed] == 1337
    print(f'All assertions passed\n')

Trying sequence indices: 0, 1
All assertions passed

Trying sequence indices: 0, 2
All assertions passed

Trying sequence indices: 0, 3
All assertions passed

Trying sequence indices: 0, 4
All assertions passed

Trying sequence indices: 0, 5
All assertions passed

Trying sequence indices: 1, 2
All assertions passed

Trying sequence indices: 1, 3
All assertions passed

Trying sequence indices: 1, 4
All assertions passed

Trying sequence indices: 1, 5
All assertions passed

Trying sequence indices: 2, 3
All assertions passed

Trying sequence indices: 2, 4
All assertions passed

Trying sequence indices: 2, 5
All assertions passed

Trying sequence indices: 3, 4
All assertions passed

Trying sequence indices: 3, 5
All assertions passed

Trying sequence indices: 4, 5
All assertions passed



In [35]:
# Generate a new solver object
solver, original_seed, seeds, next_ouputs = find_seed(sequence_length=len(known_ints))

n_solutions = 10
print(f'Looking for {n_solutions} unique original_seed values that produce our sequence')
for i in range(n_solutions):
    # Set constraints for the output
    solver.add(next_ouputs[f'next_output_{1}'] == known_ints[0])
    solver.add(next_ouputs[f'next_output_{2}'] == known_ints[1])
    
    if solver.check() == z.sat:
        solution = solver.model()[original_seed].as_long()
        solution_hex = bs.BitArray(uint=solution, length=64).hex
        print(f'Found solution #{i+1}:\t 0x{solution_hex}')
        
        # Invert the solution we found
        solver.add(solver.model()[original_seed] != original_seed)

Looking for 10 unique original_seed values that produce our sequence
Found solution #1:	 0x0000000000000539
Found solution #2:	 0x0100000000000539
Found solution #3:	 0x0200000000000539
Found solution #4:	 0x0300000000000539
Found solution #5:	 0x0002000000000539
Found solution #6:	 0x0202000000000539
Found solution #7:	 0x0102000000000539
Found solution #8:	 0x0302000000000539
Found solution #9:	 0x4000000000000539
Found solution #10:	 0x4002000000000539


In [36]:
def find_seed_nextLong(sequence_length: int, slope: int = 0x5DEECE66D, intercept: int = 0xB):
    # Define some constants
    addend = z.BitVecVal(intercept, 64)
    multiplier = z.BitVecVal(slope, 64)
    mask = z.BitVecVal((1 << 48) - 1, 64)
    
    # Note we're generating double the constraints in the sequence_length + 1
    # This is required since we'll be using seed_0 to extract the Random() instantiation value
    # Furthermore, each nextLong call consumes two outputs from next()
    next_constraints, seeds, next_outputs = make_constraints_next(n_constraints=2*sequence_length+1, gen_bits=32)
    
    # Define a symbolic variable that we'll use to get the value that instantiated Random()
    original_seed = z.BitVec('original_seed', 64)
    
    # Build a solver object
    s = z.Solver()
    
    # Build a constraint that relates seed_0 and the value used to instantiate Random()
    s.add(seeds[f'seed_0'] == (original_seed ^ multiplier) & mask)
    
    # Next let's add the constraints we've built for next()
    s.add(next_constraints)
    
    # Define symbolic variables for the output from nextLong
    # Notice: we're using a symbolic variable of type Int
    # Since nextLong does a sum on ints, it's easier to model this using Z3 arithmetic models
    # This differs from previous examples where we used BitVec objects exclusively
    # Consequently, we'll be using the conversion method BV2Int that takes a BitVec object and turns it into an Int object
    nextLong_outputs = {f'nextLong_output_{i}': z.Int(f'nextLong_output_{i}') for i in range(1, sequence_length + 1)}
    
    # Finally, let's add the constraints for nextLong
    for i, j in zip(range(1, sequence_length + 1), range(1, sequence_length * 2 + 1, 2)):
        # Notice: we've replaced the bit shift operator in the source with an enquivalent multiplication
        # Z3 doesn't support bit shift operations on Int objects
        first_next = z.BV2Int(next_outputs[f'next_output_{j}'], is_signed=True) * 2 ** 32
        second_next = z.BV2Int(next_outputs[f'next_output_{j+1}'], is_signed=True)
        s.add(nextLong_outputs[f'nextLong_output_{i}'] == first_next + second_next)
    
    # Lastly, let's return all the objects we've constructed so far   
    return s, original_seed, nextLong_outputs

In [37]:
known_longs = [-6273188232032513096, -5732460968968632244, -2153239239327503087, -1872204974168004231]

In [38]:
solver, original_seed, nextLong_outputs = find_seed_nextLong(sequence_length=len(known_longs))

# As mentioned before, we should have enough information in one long to extract the instantiation value
solver.add(nextLong_outputs[f'nextLong_output_{1}'] == known_longs[0])

# Lets take a look at our constraints before trying to solve them
print(solver)

[seed_0 == (original_seed ^ 25214903917) & 281474976710655,
 seed_1 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_0)),
 seed_2 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_1)),
 seed_3 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_2)),
 seed_4 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_3)),
 seed_5 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_4)),
 seed_6 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_5)),
 seed_7 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_6)),
 seed_8 ==
 Concat(0, 11 + 25214903917*Extract(47, 0, seed_7)),
 output1 == Extract(47, 16, seed_1),
 output2 == Extract(47, 16, seed_2),
 output3 == Extract(47, 16, seed_3),
 output4 == Extract(47, 16, seed_4),
 output5 == Extract(47, 16, seed_5),
 output6 == Extract(47, 16, seed_6),
 output7 == Extract(47, 16, seed_7),
 output8 == Extract(47, 16, seed_8),
 nextLong_output_1 ==
 If(output1 < 0,
    BV2Int(output1) - 4294967296,
    BV2Int(output1))*
 4294967296 +
 If(output2 < 0,
    BV

In [39]:
solver.check()

sat

In [40]:
solver.model()[original_seed]

1337

In [41]:
for i, known_long in enumerate(known_longs):
    calculated_long = solver.model()[nextLong_outputs[f'nextLong_output_{i+1}']].as_long()
    assert calculated_long == known_long
print('All assertions passed')

All assertions passed


In [42]:
url = 'https://10.120.129.80:8443/benchmark/weakrand-00/BenchmarkTest00086?BenchmarkTest00086=SafeText'

In [43]:
import asyncio
try:
    from pyppeteer import launch
except ModuleNotFoundError:
    !pip install pyppeteer
    from pyppeteer import launch

In [44]:
async def get_cookie(url: str) -> int:
    browser = await launch({'headless': True,
                            'args': ['--no-sandbox', '--disable-setuid-sandbox'],
                            'ignoreHTTPSErrors': True});
    page = await browser.newPage()

    await page.goto(url)
    elementList = await page.querySelectorAll('form')
    button = await elementList[0].querySelectorAll('input')
    await button[0].click()

    await page.waitForNavigation();

    cookies = await page.cookies()

    for cookie in cookies:
        if cookie['name'] == 'rememberMe00086':
            return int(cookie['value'])

In [45]:
cookie_nums = []
for i in range(1):
    cookie_nums.append(await get_cookie(url = url))
cookie_nums

[-8886403976268848760]

In [46]:
solver, original_seed, nextLong_outputs = find_seed_nextLong(sequence_length=len(known_longs))

# As mentioned before, we should have enough information in single long to extract the instantiation value
solver.add(nextLong_outputs[f'nextLong_output_{1}'] == cookie_nums[0])

# Check if satisfiable
print(f'Constraints are: {solver.check()}')

# Extract seed
print(f'Instantiation value is: {solver.model()[original_seed]}')

Constraints are: sat
Instantiation value is: 75531093912490
